In [2]:
!pip install spacy

In [3]:
!pip install spacy spacy-setfit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 958.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 88.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.6 MB/s eta 0:00:

In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Jul 31 06:53:01 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    24W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving DENV to DENV
Saving ROTVRSV to ROTVRSV
User uploaded file "DENV" with length 4054999 bytes
User uploaded file "ROTVRSV" with length 10668499 bytes


In [6]:
b1 = uploaded['ROTVRSV']
s1 = b1.decode().split('\n')
b2 = uploaded['DENV']
s2 = b2.decode().split('\n')
s1[500]

'{"DOI":"10.1099/0022-1317-73-7-1703","originalTitle":"A leucine zipper structure present in the measles virus fusion protein is not required for its tetramerization but is essential for fusion.","preprocessedText":"leucine zipper structure present measles virus fusion protein be not require tetramerization be essential fusion . it be find fusion be prevent when four heptadic leucine present motif be mutate cellular transport oligomeric state f protein be unaffected . ","label":"1"}'

In [7]:
s1[0]

'{"DOI":"10.1016/j.resp.2011.10.017","originalTitle":"\\n l-carbocisteine inhibits respiratory syncytial virus infection in human tracheal epithelial cells","preprocessedText":"l - carbocisteine inhibit respiratory syncytial virus infection human tracheal epithelial cell it be suggest l - carbocisteine may inhibit RS virus infection reduce expression icam - 1 , RS virus receptor , it may also modulate airway inflammation RSirus infection . ","label":"0"}'

In [8]:
import json, os, spacy

text_key = "preprocessedText"
inliers = []
outliers = []
for line in s1:
  if len(line) < 5:
    continue
  d = json.loads(line)
  if d["label"] == "1":
      inliers.append(d[text_key])
  else:
      outliers.append(d[text_key])

train_dataset = {
    "inlier": inliers,
    "outlier": outliers
}

tdocs = []
tlabels = []
for line in s2:
  if len(line) < 5:
    continue
  d = json.loads(line)
  tdocs.append(d[text_key])
  tlabels.append(d["label"])


In [9]:
!python -m spacy download en_core_web_trf

2023-07-31 06:57:40.076683: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.3/460.3 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.8/190.8 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.31.0
    Uninstalling transformers-4.31.0:
      Successfully uninstalled transformers-4.31.0
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


In [10]:
!pip install spacy-transformers

In [16]:
import spacy_transformers
# Load the spaCy language model:
nlp = spacy.load("en_core_web_trf")

# Add the "text_categorizer" pipeline component to the spaCy model, and configure it with SetFit parameters:
nlp.add_pipe("text_categorizer", config={
    "pretrained_model_name_or_path": "menadsa/S-BioELECTRA",
    "setfit_trainer_args": {
        "train_dataset": train_dataset
    }
})

print("{}\n{}".format(tdocs[0],nlp(tdocs[0]).cats))












Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/ProdicusII_ZeroShotBioNER were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at /root/.cache/torch/sentence_transformers/ProdicusII_ZeroShotBioNER and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
model_head.pkl not f

Generating Training Pairs:   0%|          | 0/20 [00:00<?, ?it/s]

***** Running training *****
  Num examples = 919600
  Num epochs = 1
  Total optimization steps = 57475
  Total train batch size = 16


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/57475 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [13]:
p = []
ctr = 0
for tdoc in tdocs:
  ctr = ctr + 1
  if ctr % 100 == 0:
    print(ctr)
  p.append(nlp(tdoc).cats.get('inlier'))

import numpy as np
#Fish for best cut
for cut in np.arange(0.25, .95, .05):
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    N = len(p)
    for i in range(N):
        tru = tlabels[i] == '1'
        prd = p[i] > cut
        if tru and prd == tru:
            tp = tp + 1
        if tru and prd != tru:
            fn = fn + 1
        if tru == False and prd == tru:
            tn = tn + 1
        if tru == False and prd != tru:
            fp = fp + 1
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    f1_score = 2 * (precision * recall) / (precision + recall)
    print("cut: {}, acc: {}, prec: {}, rec: {}, f1: {}".format(cut, (tp+tn)/N, precision, recall, f1_score))

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
cut: 0.25, acc: 0.9269501094596152, prec: 0.8370339338081274, rec: 0.8907712884529648, f1: 0.8630669546436286
cut: 0.3, acc: 0.9269501094596152, prec: 0.8373165618448637, rec: 0.8903254569772626, f1: 0.8630077787381157
cut: 0.35, acc: 0.9269501094596152, prec: 0.8373165618448637, rec: 0.8903254569772626, f1: 0.8630077787381157
cut: 0.39999999999999997, acc: 0.9270653301071552, prec: 0.8376677852348994, rec: 0.8903254569772626, f1: 0.863194294359196
cut: 0.44999999999999996, acc: 0.9272957714022353, prec: 0.8383711167086482, rec: 0.8903254569772626, f1: 0.8635675675675676
c